In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_syste

In [2]:
# 1. 데이터 로드 및 확인
from datasets import load_dataset

In [3]:
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/733k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/6.36M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [4]:
dataset

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})

In [5]:
dataset['train'][3]

{'text': ' Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . \n'}

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [7]:
model_name = 'gpt2'
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [8]:
# GPT-2 에는 기본 pad_token이 없으므로, eos_token 으로 설정
tokenizer.pad_token = tokenizer.eos_token

In [9]:
# 데이터셋의 데이터를 토크나이징 하는 함수
def tokenize_function(examples) :
  tokens = tokenizer(examples['text'], padding="max_length", truncation=True, max_length= 512)
  # Trainer 가 인식할 수 있는 형식으로 변환
  # GPT-2 는 자기 자신을 예측하는 방식으로 학습했다.(자기 회귀 모델)
  return {'input_ids' : tokens['input_ids'], 'attention_mask':tokens['attention_mask'], 'labels' : tokens['input_ids']}

In [10]:
dataset

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})

In [11]:
train_dataset = dataset['train']

In [12]:
test_dataset = dataset['test']

In [13]:
train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=['text'])

Map:   0%|          | 0/36718 [00:00<?, ? examples/s]

In [14]:
test_dataset = test_dataset.map(tokenize_function, batched=True, remove_columns=['text'])

Map:   0%|          | 0/4358 [00:00<?, ? examples/s]

In [15]:
from transformers import TrainingArguments, Trainer

In [16]:
# 학습 설정
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    num_train_epochs=1,
    weight_decay=0.01,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_dir='./logs'
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [17]:
# Trainer 생성
trainer = Trainer(
    model= model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [18]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: blockenters (blockenters-mycompany) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss
1,0.429400,0.412187


TrainOutput(global_step=2295, training_loss=0.439225207226988, metrics={'train_runtime': 1159.3231, 'train_samples_per_second': 31.672, 'train_steps_per_second': 1.98, 'total_flos': 9594120830976000.0, 'train_loss': 0.439225207226988, 'epoch': 1.0})

In [29]:
prompt = "The future of AI is"
inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
print(inputs)
print()
outputs = model.generate(**inputs)
print(outputs)
tokenizer.decode(outputs[0], skip_special_tokens=True)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


{'input_ids': tensor([[ 464, 2003,  286, 9552,  318]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1]], device='cuda:0')}

tensor([[  464,  2003,   286,  9552,   318,  8627,   764,   383,  9552,  2055,
           318,  9086,   656,   734, 11665,  1058,   883,   508,  1975,   326,
          9552,   318,   257,   922,  2126]], device='cuda:0')


'The future of AI is uncertain . The AI community is divided into two camps : those who believe that AI is a good idea'

In [ ]:
# 문장 분류
# 데이터셋 : ag_news
# 뉴스 기사를 4개의 카테고리로 분류하는 데이터셋 입니다.
# 0 : 세계뉴스
# 1 : 스포츠
# 2 : 비즈니스
# 3 : 과학/기술

In [30]:
dataset = load_dataset("ag_news")
train_dataset = dataset['train'].shuffle().select( range(1000) )  # 1000개 데이터를 랜덤으로 가져온다.
test_dataset = dataset['test'].shuffle().select( range(200) )   # 200개 데이터를 랜덤으로 가져온다.

README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [31]:
train_dataset[0]

{'text': 'Bangladesh paralysed by strikes Opposition activists have brought many towns and cities in Bangladesh to a halt, the day after 18 people died in explosions at a political rally.',
 'label': 0}

In [32]:
# 레이블 목록 확인하고 싶다
dataset['train'].features['label']

ClassLabel(names=['World', 'Sports', 'Business', 'Sci/Tech'], id=None)

In [ ]:
## 파인 튜닝 에 사용할 모델과 토크나이저를 가져온다.

In [33]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

In [34]:
model_name = "bert-base-uncased"

In [63]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [64]:
# 데이터 전처리 : 토크나이징
def tokenize_function(examples) :
  return tokenizer(examples['text'], padding=True, truncation=True, max_length=128, return_tensors='pt')

In [65]:
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [66]:
train_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1000
})

In [67]:
train_dataset[0]

{'text': 'Bangladesh paralysed by strikes Opposition activists have brought many towns and cities in Bangladesh to a halt, the day after 18 people died in explosions at a political rally.',
 'label': 0,
 'input_ids': [101,
  7269,
  11498,
  2135,
  6924,
  2011,
  9326,
  4559,
  10134,
  2031,
  2716,
  2116,
  4865,
  1998,
  3655,
  1999,
  7269,
  2000,
  1037,
  9190,
  1010,
  1996,
  2154,
  2044,
  2324,
  2111,
  2351,
  1999,
  18217,
  2012,
  1037,
  2576,
  8320,
  1012,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'token_type_ids': [0,
  0,
  0,
  0,

In [68]:
# 학습 설정
training_args = TrainingArguments(
    output_dir='./results2',
    evaluation_strategy='epoch',
    num_train_epochs=4,
    weight_decay=0.01,
    per_device_train_batch_size=8,
    logging_dir='./logs',
    save_strategy='epoch'
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [69]:
# 트레이너 생성
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = test_dataset
)

In [70]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,0.600986
2,No log,0.345563
3,No log,0.383955
4,0.296800,0.422205


TrainOutput(global_step=500, training_loss=0.29681332397460936, metrics={'train_runtime': 44.8711, 'train_samples_per_second': 89.144, 'train_steps_per_second': 11.143, 'total_flos': 263115780096000.0, 'train_loss': 0.29681332397460936, 'epoch': 4.0})

In [71]:
# 에포크 2가 가장 좋은 모델이므로,
# 이 모델을 허깅페이스에 업로드 한다.

In [75]:
from huggingface_hub import notebook_login
notebook_login()

In [72]:
from huggingface_hub import HfApi
api = HfApi()
my_model_name = 'blockenters/bert-agnews'
api.create_repo(repo_id=my_model_name, private=True, exist_ok=True)

In [137]:
# 특정 체크포인트 경로
checkpoint_path = "/content/results2/checkpoint-250"

# 최종 모델 저장 경로 (이 폴더를 업로드)
final_model_path = "blockenters/bert-agnews"

# 체크포인트에서 모델 불러오기
model = AutoModelForSequenceClassification.from_pretrained(checkpoint_path)
model.id2label = {0:'World', 1:'Sports', 2:'Business', 3:'Sci/Tech'}
model.label2id = {'World':0, 'Sports':1, 'Business':2, 'Sci/Tech':3}

# 모델을 업로드
model.push_to_hub(final_model_path)
tokenizer.push_to_hub(final_model_path)

No files have been modified since last commit. Skipping to prevent empty commit.
No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/blockenters/bert-agnews/commit/7a8db837c1d12b6418a56019d74e034aa60e0b27', commit_message='Upload tokenizer', commit_description='', oid='7a8db837c1d12b6418a56019d74e034aa60e0b27', pr_url=None, repo_url=RepoUrl('https://huggingface.co/blockenters/bert-agnews', endpoint='https://huggingface.co', repo_type='model', repo_id='blockenters/bert-agnews'), pr_revision=None, pr_num=None)

In [133]:
text = "Apple releases a new iPhone"

In [134]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
inputs = tokenizer(text, return_tensors='pt').to(device)

In [135]:
inputs

{'input_ids': tensor([[  101,  6207,  7085,  1037,  2047, 18059,   102]], device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [141]:
model.to(device)
ogits = model(**inputs).logits # Perform the forward pass with the model on the correct device
predicted_labels = torch.argmax(logits, dim=-1).tolist()

In [147]:
config.id2label[ predicted_labels[0] ]

'Sci/Tech'

In [145]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
import torch

# Specify the model repository and subfolder
model_name = "blockenters/bert-agnews-final"

# # Now you can use the model for inference
text = "Success in business comes from solving real problems. Focus on customer needs, build strong relationships, and innovate continuously. Adapt to change, stay persistent, and always deliver value."

# Alternatively, use the pipeline with the loaded model and tokenizer
pipe = pipeline('text-classification', model=model_name, tokenizer=tokenizer)
print(pipe(text))

Device set to use cuda:0


[{'label': 'LABEL_3', 'score': 0.929456889629364}]
